# RGB Baseline Models (EuroSAT)

This notebook trains baseline RGB image classification models on the EuroSAT dataset using the fixed train/validation/test splits defined in `01_dataset_preparation_and_splits.ipynb`

The goal is to establish a clean performance baseline before introducing multispectral inputs or physics-aware inductive biases.

---

In [1]:
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, models
from torchvision.datasets import EuroSAT

# Project root (robust to running from notebooks/)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

DATA_DIR = PROJECT_ROOT / "data"
SPLITS_DIR = PROJECT_ROOT / "splits"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
ARTIFACTS_DIR.mkdir(exist_ok=True)

In [2]:
splits = np.load(SPLITS_DIR / "eurosat_rgb_splits.npz")

train_idx = splits["train_idx"]
val_idx   = splits["val_idx"]
test_idx  = splits["test_idx"]

print(len(train_idx), len(val_idx), len(test_idx))

21600 2700 2700


In [3]:
rgb_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

ds_rgb = EuroSAT(
    root=str(DATA_DIR / "eurosat_rgb"),
    download=False,   # already downloaded
    transform=rgb_transform
)

train_ds = Subset(ds_rgb, train_idx)
val_ds   = Subset(ds_rgb, val_idx)
test_ds  = Subset(ds_rgb, test_idx)

In [4]:
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=128, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=128, shuffle=False, num_workers=0)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(ds_rgb.classes)

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [7]:
def train_one_epoch(model, loader):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)

    return total_loss / total, correct / total

In [13]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)

    return correct / total

In [14]:
history = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": [],
}
best_val_acc = -1.0
best_epoch = None
best_state = None

In [15]:
epochs = 5

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_acc = evaluate(model, val_loader)  # <-- float

    history["train_loss"].append(float(train_loss))
    history["train_acc"].append(float(train_acc))
    history["val_acc"].append(float(val_acc))

    print(
        f"Epoch {epoch}/{epochs} | "
        f"train loss {train_loss:.4f} acc {train_acc:.4f} | "
        f"val acc {val_acc:.4f}"
    )

    if val_acc > best_val_acc:
        best_val_acc = float(val_acc)
        best_epoch = epoch
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

print("Best epoch:", best_epoch, "| Best val acc:", best_val_acc)

Epoch 1/5 | train loss 0.3625 acc 0.8737 | val acc 0.7693
Epoch 2/5 | train loss 0.2932 acc 0.8988 | val acc 0.4733
Epoch 3/5 | train loss 0.2454 acc 0.9135 | val acc 0.7519
Epoch 4/5 | train loss 0.2164 acc 0.9235 | val acc 0.7119
Epoch 5/5 | train loss 0.1729 acc 0.9412 | val acc 0.6559
Best epoch: 1 | Best val acc: 0.7692592592592593


In [16]:
model.load_state_dict(best_state)

test_acc = evaluate(model, test_loader)
print(f"FINAL (best val) | epoch={best_epoch} | val={best_val_acc:.4f} | test={test_acc:.4f}")

FINAL (best val) | epoch=1 | val=0.7693 | test=0.7570


In [17]:
import json
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
ARTIFACTS_DIR.mkdir(exist_ok=True)

WEIGHTS_FILE = ARTIFACTS_DIR / "rgb_resnet18_best.pt"      
RESULTS_FILE = ARTIFACTS_DIR / "rgb_resnet18_results.json" 

torch.save(model.state_dict(), WEIGHTS_FILE)

results = {
    "best_epoch": int(best_epoch),
    "best_val_acc": float(best_val_acc),
    "test_acc": float(test_acc),
    "history": history,
    "epochs": int(epochs),
}
with open(RESULTS_FILE, "w") as f:
    json.dump(results, f, indent=2)

print("Saved weights:", WEIGHTS_FILE)
print("Saved results:", RESULTS_FILE)

Saved weights: C:\Users\ishaa\OneDrive\Desktop\Projects\eurosat-physics-aware-image-classification\artifacts\rgb_resnet18_best.pt
Saved results: C:\Users\ishaa\OneDrive\Desktop\Projects\eurosat-physics-aware-image-classification\artifacts\rgb_resnet18_results.json
